In [1]:
# sort outputs by absolute sentence-level BLEU improvement over unconstrained

# print source, hyp1, hyp2, constraints

In [2]:
from machine_translation.evaluation import sentence_level_bleu, mteval_13

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5103)
/home/chris/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
DEBUG:root:test


In [3]:
LANG_PAIR = 'EN-PT'

if LANG_PAIR == 'EN-DE':
    source_file = '/media/1tb_drive/parallel_data/en-de/dev_sets/wmt16_dev/tokenized/newstest2013.en.bpe.tok'
    ref_file = '/media/1tb_drive/parallel_data/en-de/dev_sets/wmt16_dev/tokenized/newstest2013.de.bpe.tok'

    baseline_hyps_file = '/home/chris/projects/constrained_decoding/experiments/en-de-primt-gpu/primt.translations.0'
    constrained_hyps_file = '/home/chris/projects/constrained_decoding/experiments/en-de-primt-gpu/primt.translations.3'
    constraints_file = '/home/chris/projects/constrained_decoding/experiments/en-de-primt-gpu/cycle_constraints.2.json'

if LANG_PAIR == 'EN-FR':
    source_file = '/media/1tb_drive/parallel_data/en-fr/phrasal_acl/devtest/newstest2014.enfr.en.bpe'
    ref_file = '/media/1tb_drive/parallel_data/en-fr/phrasal_acl/devtest/newstest2014.enfr.fr.bpe'

    baseline_hyps_file = '/home/chris/projects/constrained_decoding/experiments/en-fr-primt-gpu/primt.translations.0'
    constrained_hyps_file = '/home/chris/projects/constrained_decoding/experiments/en-fr-primt-gpu/primt.translations.3'
    constraints_file = '/home/chris/projects/constrained_decoding/experiments/en-fr-primt-gpu/cycle_constraints.2.json'

if LANG_PAIR == 'EN-PT':
    source_file = '/home/chris/projects/constrained_decoding/proto/autodesk_constrained_decoding_corpus/en-pt/autodesk.dev.1000.en.bpe'
    ref_file = '/home/chris/projects/constrained_decoding/proto/autodesk_constrained_decoding_corpus/en-pt/autodesk.dev.1000.pt.bpe'

    baseline_hyps_file = '/home/chris/projects/constrained_decoding/experiments/en-pt-primt-autodesk/run_1/primt.translations.0'
    constrained_hyps_file = '/home/chris/projects/constrained_decoding/experiments/en-pt-primt-autodesk/run_1/primt.translations.3'
    constraints_file = '/home/chris/projects/constrained_decoding/experiments/en-pt-primt-autodesk/run_1/cycle_constraints.2.json'



In [4]:
def sentence_level_bleu(src, ref, trg):
    return mteval_13(src, ref, trg)

In [5]:
# Sentence-level BLEU
baseline_scores = sentence_level_bleu(source_file, ref_file, baseline_hyps_file)
constrained_scores = sentence_level_bleu(source_file, ref_file, constrained_hyps_file)


DEBUG:root:perl /home/chris/projects/neural_mt/scripts/scoring/wrap-xml-modified.perl en /home/chris/projects/constrained_decoding/proto/autodesk_constrained_decoding_corpus/en-pt/autodesk.dev.1000.en.bpe nmt src< /home/chris/projects/constrained_decoding/proto/autodesk_constrained_decoding_corpus/en-pt/autodesk.dev.1000.en.bpe
DEBUG:root:perl /home/chris/projects/neural_mt/scripts/scoring/wrap-xml-modified.perl de /tmp/tmpwd_hTm nmt ref< /home/chris/projects/constrained_decoding/proto/autodesk_constrained_decoding_corpus/en-pt/autodesk.dev.1000.pt.bpe
DEBUG:root:perl /home/chris/projects/neural_mt/scripts/scoring/wrap-xml-modified.perl de /tmp/tmpwd_hTm nmt tst< /home/chris/projects/constrained_decoding/experiments/en-pt-primt-autodesk/run_1/primt.translations.0
DEBUG:root:perl /home/chris/projects/neural_mt/scripts/scoring/mteval-v13a.pl -r /tmp/tmpXcpMYk -s /tmp/tmpwd_hTm -t /tmp/tmpvBec1i -b -d 2
DEBUG:root:perl /home/chris/projects/neural_mt/scripts/scoring/wrap-xml-modified.perl 

In [6]:
len(baseline_scores)
len(constrained_scores)

1000

In [7]:
score_diffs = [(idx, s2 - s1) for idx, (s1, s2) in enumerate(zip(baseline_scores, constrained_scores))]
most_improved = sorted(score_diffs, key=lambda x: x[1], reverse=True)

In [8]:
import codecs
import json

In [9]:
def get_lines(filename):
    with codecs.open(filename, encoding='utf8') as inp:
        return inp.read().strip().split('\n')

In [10]:
source_lines = get_lines(source_file)
ref_lines = get_lines(ref_file)
baseline_hyp_lines = get_lines(baseline_hyps_file)
constrained_hyp_lines = get_lines(constrained_hyps_file)
constraints = json.loads(codecs.open(constraints_file, encoding='utf8').read())

In [11]:
import difflib

# time per segment given a dict of segments 
def get_editdistance(hyp, ref, char_level=True):

    matcher = difflib.SequenceMatcher(isjunk=None, a=hyp, b=ref)
    return 1. - matcher.ratio()

In [12]:
for idx, diff in most_improved[:3000]:
    source_len = len(source_lines[idx].split())
    target_len = len(ref_lines[idx].split())
    edit_distance = get_editdistance(baseline_hyp_lines[idx], ref_lines[idx])
#     print(edit_distance)
    if (target_len > 10
    and target_len < 30
    and len(constraints[idx]) > 1
    and edit_distance >= 0.5):
        print(u'S1: {} S2: {}'.format(baseline_scores[idx], constrained_scores[idx]))
        print(u'Source: {}'.format(source_lines[idx]))
        print(u'Hyp1: {}'.format(baseline_hyp_lines[idx]))
        print(u'Constraints: {}'.format(constraints[idx]))
        print(u'Hyp2: {}'.format(constrained_hyp_lines[idx]))
        print(u'Ref: {}\n'.format(ref_lines[idx]))

        
# string alignment between hyp1, hyp2, and 

S1: 0.8546 S2: 0.8338
Source: Ultimate : In@@ cludes all Pre@@ mium edition features , plus Ali@@ as Design advanced sur@@ facing capabilities .
Hyp1: In@@ dos todos os acontecimentos da edição dos Pre@@ tor@@ ais , mais Ali@@ as da capacidade de sobrevivência da cidade .
Constraints: [[u'recursos', u'avan\xe7ados', u'para'], [u'O', u'Ultimate', u'inclui'], [u'todos', u'os']]
Hyp2: In@@ pondo todos os recursos <UNK> para a edição , mais Ali@@ as da capacidade de sobrevivência da cidade . O Ultimate inclui as capacidades
Ref: O Ultimate inclui todos os recursos da edição Pre@@ mium , além de recursos avançados para superfícies do Ali@@ as Design .

S1: 0.9229 S2: 0.8756
Source: Pan and Ro@@ tate the Rail line .
Hyp1: E o Ro@@ dr@@ ato a linha de comboios .
Constraints: [[u'A@@', u'plique', u'panor\xe2mica'], [u'panor\xe2mica', u'e', u'rota\xe7\xe3o'], [u'rota\xe7\xe3o', u'\xe0', u'linha']]
Hyp2: A@@ plique <UNK> <UNK> e <UNK> <UNK> <UNK> linha .
Ref: A@@ plique panorâmica e rotação à li

In [13]:
edit_distances = [(idx, get_editdistance(source_lines[idx], ref_lines[idx])) for idx in range(len(source_lines))]

closest_to_perfect = sorted(edit_distances, key=lambda x: x[1], reverse=True)

for idx, diff in closest_to_perfect[:3000]:
    source_len = len(source_lines[idx].split())
    target_len = len(ref_lines[idx].split())
    edit_distance = get_editdistance(baseline_hyp_lines[idx], ref_lines[idx])

#     edit_distance = get_editdistance(source_lines[idx], ref_lines[idx])
#     print(edit_distance)
    if (target_len > 10
    and target_len < 30
    and len(constraints[idx]) > 1
    and edit_distance >= 0.3
    and constrained_scores[idx] == 0.
    and u'UNK' not in constrained_hyp_lines[idx]):
        print(u'S1: {} S2: {}'.format(baseline_scores[idx], constrained_scores[idx]))
        print(u'Source: {}'.format(source_lines[idx]))
        print(u'Hyp1: {}'.format(baseline_hyp_lines[idx]))
        print(u'Constraints: {}'.format(constraints[idx]))
        print(u'Hyp2: {}'.format(constrained_hyp_lines[idx]))
        print(u'Ref: {}\n'.format(ref_lines[idx]))

        
# string alignment between hyp1, hyp2, and 

S1: 0.7555 S2: 0.0
Source: To specify the route the selected wires should take :
Hyp1: Ac@@ ende a rota que os fios seleccionados deveriam tomar .
Constraints: [[u'selec@@', u'ion@@', u'ados'], [u'devem', u'seguir', u':'], [u'Para', u'especificar']]
Hyp2: Para especificar a rota que os fios selec@@ ion@@ ados devem seguir :
Ref: Para especificar a rota que os fios selec@@ ion@@ ados devem seguir :

S1: 0.9528 S2: 0.0
Source: Mo@@ dif@@ y drain@@ age features by selecting them individually .
Hyp1: - Já temos as características de extracção de idade , com eles individualmente .
Constraints: [[u'drenagem', u'ao', u'selec@@'], [u'Modi@@', u'fique', u'os'], [u'recursos']]
Hyp2: Modi@@ fique os recursos de drenagem ao selec@@ ion@@ á-los individualmente .
Ref: Modi@@ fique os recursos de drenagem ao selec@@ ion@@ á-los individualmente .

S1: 1.0 S2: 0.0
Source: GU@@ ID@@ -B@@ EE@@ 02@@ D@@ 4@@ 5-0@@ 9@@ 15-@@ 4@@ 1@@ D@@ 8-@@ A@@ 09@@ E-@@ D@@ 0@@ E@@ 4@@ FA@@ 7@@ A@@ 30@@ 23
Hyp1: Tradução 